# Bitcoin's ellitpic curve, private keys and public keys

In [1]:
from finite_field import FiniteFieldElement
from ec import EllipticCurve, ECPoint
import math
import importlib
import warnings
warnings.filterwarnings('ignore')

Up until now we have learned about finite fields, elliptitc curves over real numbers, and the combination of the two to get elliptic curves over finite fields. These are the fundamental building blocks of elliptic curve cryptography (ECC). Before we dive into how elliptic curve cryptography, especially signing and verifying messages, works, let's have a look at the elliptic curve that Bitcoin uses and what private and public keys actually are and how they work.

## Bitcoin's elliptic curve

The cryptographic (elliptic) curve that Bitcoin uses is called "secp256k1". This curve is defined by the following parameters:
- $a=0, b=7$, so that the curve equation is: $y^2 = x^3 + 7$
- Prime order of the finite field that the curve is over: $p = 2^{256} - 2^{32} - 977$
- Generator point, $G$, defined by the following x- and y-coordinates:
    - $G_x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798$
    - $G_y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8$
- Order of $G$: $n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141$

The numbers for $G_x, G_y and n$ are in hexadecimal representation and in decimal look like this:

In [2]:
G_x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
G_y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
print(f'G_x: {G_x}')
print(f'G_y: {G_y}')
print(f'n: {n}')

G_x: 55066263022277343669578718895168534326250603453777594175500187360389116729240
G_y: 32670510020758816978083085130507043184471273380659243275938904335757337482424
n: 115792089237316195423570985008687907852837564279074904382605163141518161494337


These are all just really large integers. The same goes for $p$, which is really close to $2^{256}$:

In [3]:
p = 2 ** 256 - 2 ** 32 - 977
print(f'p: {p}')

p: 115792089237316195423570985008687907853269984665640564039457584007908834671663


Interestingly, $n$ is a lot smaller than $p$ in terms of any order of magnitude we are used to from every day life (millions, billions and even trillions (US government debt for example)) but both of these numbers are so large that their absolute difference is just a rounding error in relative terms:

In [4]:
print(f'p - n = {p - n}, which is {round((p - n) / (10 ** (len(str(p - n)) - 1)), 3)} * 10^{len(str(p - n)) - 1}')
print(f'p / n : {p/n}')

p - n = 432420386565659656852420866390673177326, which is 4.324 * 10^38
p / n : 1.0


The difference between $p$ and $n$ is astronomical in terms of orders of magnitude that we are used to but because $p$ and $n$ are both integers of an order of magnitude of 77, a difference with an order of magnitude of 38 is practically no difference.<br><br>
Remember that $n$ is the order of the generator point, which means that there are $n-1$ points on the elliptic curve. The order of the generator point used for Bitcoin's elliptic curve is close to $2^{256}$. Because $2^{256}$, which is roughly $10^{77}$ is not a number that most people can comprehend, here are some comparisons that put it into perspective (source: Programming Bitcoin by Jimmy Song):
- Number of atoms in and on Earth: $~ 10^{50}$
- Number of atoms in the solar system: $~ 10^{57}$
- Number of atoms in the Milky Way: $~ 10^{68}$
- Number of atoms in the observable universe: $~ 10^{80}$
- A trillion computers doing a trillion computations every trillionth of a second for a trillion yars is still less than $10^{56}$ computations

We will see why this is so important below.

## Private and Public Keys

In essence, the private key, $d$ is just a large integer in the range $[0, ..., n-1]$ that is securely and randomly generated. The public key, $Q$, is simply the point on the elliptic curve that is the result of point multiplying the generator point, $G$, with the private key:
$$Q = dG$$ 
As we learned in the lesson on elliptic curves over finite fields, if we know $d$ and $G$ it's really easy to compute $Q$ but if we only know $G$ and $Q$, it's practically impossible to solve for $d$. This is the Elliptic Curve Discrete Logarithm Problem (ECDLP) and it's the reason why elliptic curve cryptography on the Bitcoin blockchain is so secure. With modern supercomputers no one will be able to backsolve for your private key, even if they have knowledge of your public key and the generator point (both of which are publicly available). And because knowledge of private keys is eqivalent to ownership of Bitcoin, as long as your private keys are truly secret and uncompromised, your Bitcoin is secured by unbreakable mathematics. Truly incredible. <br><br>
Let's look at an example private key and what public key it results in:

In [5]:
# Define the Bitcoin curve
a = FiniteFieldElement(0, p)
b = FiniteFieldElement(7, p)
btc_curve = EllipticCurve(a, b)

# Create the generator point
p = 2 ** 256 - 2 ** 32 - 977
G_x = FiniteFieldElement(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798, p)
G_y = FiniteFieldElement(0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8, p)
generator = ECPoint(G_x, G_y, btc_curve)

# Define a private key
private_key = int.from_bytes(b'Satoshi Nakamoto', 'big')
print(f'Private key as integer: {private_key}')

# Compute the public key
public_key = private_key * generator
print(f'Public key: {public_key.x.num, public_key.y.num}')

Private key as integer: 110831938034982959267213124394594301039
Public key: (9189960675685860389423310802391916115592466696906485832633579783307896203399, 91239177961079355031999187374243160150596276681847926269292121519844954103361)


We can even check the address of this public key on a blockchain explorer such as https://blockexplorer.one/bitcoin/mainnet for the mainnet and https://blockexplorer.one/bitcoin/testnet for the testnet. Here are the addresses:

In [6]:
print(f'Mainnet address of public key: {public_key.address()}')
print(f'Testnet address of public key: {public_key.address(testnet=True)}')

Mainnet address of public key: bc1qcnym9xcuelypja2lh620k350kwm9evr5ys68wz
Testnet address of public key: tb1qcnym9xcuelypja2lh620k350kwm9evr5wkp543


## Why $2^{256}$ is important for the public key cryptography

The fact that the order, $n$, of the generator point is close to $2^{256}$ is important because the private key that you use to generate your public Bitcoin address (which is where your Bitcoin is stored) can be any random integer from $1$ to $n-1$. This means that there are so many possible values that you could've used to compute your public key that it's impossible to try all the possible values and thereby "steal" your keys and your Bitcoin. Finding a private key for a public key is like asking someone to pick a random atom from all the atoms in a billion galaxies, then scrambling those atoms up completely randomly, also choosing an atom at random and ending up with the same atom. Good luck...

## Summary

In this lesson we learned:
- What the parameters of the Bitcoin elliptic curve are.
- What private and public keys actually are.
- How you compute a public key from a given private key.
- What the order of the generator point is and why it's astronomical size makes ECC so secure.